# Langchain -> Secao03


In [ ]:
# !pip install --upgrade langchain langchain-community langchain-huggingface langchainhub langchain_chroma transformers tokenizers bitsandbytes accelerate torch

In [ ]:
import torch
import os
import getpass

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain_huggingface import HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

## Modelos

### Phi-3-mini-4k-instruct

In [ ]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 1000,
    do_sample = True,
    repetition_penalty = 1.1,
    return_full_text = False,
)


In [ ]:
llm = HuggingFacePipeline(pipeline = pipe)

input = "Quem foi a primeira pessoa no espaço?"
output = llm(input)
print(output)

### Meta Llama3-8b

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)


model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 1000,
    do_sample = True,
    repetition_penalty = 1.1,
    return_full_text = False,
)

llm = HuggingFacePipeline(pipeline = pipe)

In [ ]:
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [ ]:
system_prompt = "Você é um assistente e está respondendo perguntas gerais"
user_prompt = "Qual foi a primeira linguagem de programação?"

prompt_template = template.format(system_prompt = system_prompt , user_prompt = user_prompt)
print(prompt_template)

output = llm.invoke(prompt_template)
print(output)

### Modelos de Chat

In [ ]:
from langchain_core.messages import (HumanMessage , SystemMessage)
from langchain_huggingface import ChatHuggingFace

In [ ]:
msgs = [
        SystemMessage(content = "Você é um assistente e está respondendo perguntas gerais"),
        HumanMessage(content = "Traduza o texto para o espanhol: Texto: olá!")
]

chat_model = ChatHuggingFace(llm = llm)

In [ ]:
model_template = tokenizer.chat_template
chat_model._to_chat_prompt(msgs)

In [ ]:
res = chat_model.invoke(msgs)
print(res.content)

## Prompt Templates

### ChatPromptTemplates


In [ ]:
from langchain_core.prompts import StringPromptTemplate

prompt_template = PromptTemplate.from_template("Escreva um poema sobre {topic}")
prompt_template.invoke({"topic" : "Abacates"})

### ChatPromptTemplates


In [ ]:
from langchain_core.prompts import ChatPromptTemplate 

prompt = ChatPromptTemplate.from_messages([
    ("system" , "Você é um assistente e está respondendo perguntas gerais"),
    ("user" , "Explique-me em uma frase o conceito de {topic}")
])

prompt.invoke({"topic" : "IA"})

In [ ]:
user_prompt = "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva no máximo {tamanho}."
prompt = PromptTemplate.from_template(template.format(system_prompt = system_prompt , user_prompt = user_prompt))
prompt.invoke({"topic" : "IA" , "tamanho" : "1 frase"})